In [1]:
using Plots, ForwardDiff, Interact, Printf, LinearAlgebra

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-15126495339563897354\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-13512584888860429279\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-15126495339563897354\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
# check if matrix is symmetric
function isSymmetric(M)
    (m,n) = size(M)   
    if m != n
        return false
    end
    for i in 1:m
        for j in 1:n
            if M[i,j] != M[j,i]
                return false
            end
        end
    end
    return true
end

# one iterate of cholesky factorization
function CholeskyIterate(M)
    (n,~) = size(M)
    
    ϵ = .1;
    tol = 1e-4
    
    R = similar(M)
    A = similar(M)
    log_PD = true

    if abs(M[1,1]) <= tol      
        log_PD = false #set α = ϵ
        α = ϵ; 
        w = M[1,2:end]
        R = [α w'/α ; zeros(n-1,1) I(n-1)]
        A = [1 zeros(1,n-1);zeros(n-1,1) M[2:end,2:end] - w*w' / (α^2)]
    else
        α = √(M[1,1])
        w = M[1,2:end]
        R = [α w'/α ; zeros(n-1,1) I(n-1)]
        A = [1 zeros(1,n-1);zeros(n-1,1) M[2:end,2:end] - w*w' / (α^2)]    
    end
    return log_PD, R, A    
end

function CholeskyFull(M)
    tol = 1e-4
    
    if isSymmetric(M)
        (n,~) = size(M) 

        Rt = I(n)
        for i in 1:n-1
            (log_PD, R, M) = CholeskyIterate(M[i:end,i:end])
            if log_PD
                Rt = [I(i-1) zeros(i-1, n-i+1); zeros(n-i+1, i-1 ) R] * Rt
            else
                @printf("Matrix is not positive definite\n")
                return nothing
            end                
        end   
        
        if abs(M[2,2]) <= tol
            @printf("Matrix is not positive definite\n")
            return nothing
        else
            Rt[n,n] = Rt[n,n] * √(M[2,2])
        end

        return Rt
    else
        @printf("Matrix is not symmetric\n")
        return nothing
    end       
end

CholeskyFull (generic function with 1 method)

In [3]:
x1 = randn(3)
x2 = randn(3)
x3 = randn(3)

X = x1*x1' + x2*x2';
@printf("Cholesky:\n")
K1 = CholeskyFull(X)


# @show(norm(K1'K1 - X));


Cholesky:
Matrix is not positive definite


In [4]:
# modified Cholesky
function ModifiedCholeskyFull(M)
    ϵ = .1
    tol = 1e-4;
    
    if isSymmetric(M)
        (n,~) = size(M) 

        Rt = I(n)
        for i in 1:n-1
            (log_PD, R, M) = CholeskyIterate(M[i:end,i:end])
            if log_PD
                Rt = [I(i-1) zeros(i-1, n-i+1); zeros(n-i+1, i-1 ) R] * Rt
            else
                @printf("Using moified Cholesky step\n")
                Rt = [I(i-1) zeros(i-1, n-i+1); zeros(n-i+1, i-1 ) R] * Rt
            end                
        end   
        
        if abs(M[2,2]) <= tol
            @printf("Using moified Cholesky step\n")
            Rt[n,n] = Rt[n,n] * ϵ
        else
            Rt[n,n] = Rt[n,n] * √(M[2,2])
        end
        
        return Rt
            
    else
        @printf("Matrix is not symmetric\n")
        return nothing
    end       
end


ModifiedCholeskyFull (generic function with 1 method)

In [5]:
x1 = randn(3)
x2 = randn(3)
x3 = randn(3)

X = x1*x1' + x2*x2';
@printf("Cholesky:\n")
K1 = ModifiedCholeskyFull(X)

@show(norm(K1'K1 - X));


Cholesky:
Using moified Cholesky step
norm(K1' * K1 - X) = 0.01000000000000223
